In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("Supply_chain_data.csv")
df = df.sort_values(by=["Transaction ID", "Timestamp"])

# Initialize battery column
df['battery_level'] = np.nan

# Loop per tracker / shipment
for tx in df['Transaction ID'].unique():
    subset = df[df['Transaction ID'] == tx]
    n = len(subset)
    start = np.random.uniform(85, 100)  # initial battery
    battery = [start]

    for i in range(1, n):
        prev = battery[-1]
        stage = subset.iloc[i]['Stage']
        temp = subset.iloc[i]['Temperature']

        # Base drain
        if "Warehouse" in stage:
            drain = np.random.uniform(0.01, 0.1)
        elif "Transport" in stage or "Delivery" in stage or "Retailer" in stage:
            drain = np.random.uniform(0.2, 0.8)
        else:
            drain = np.random.uniform(0.05, 0.3)

        # Temperature effect
        if temp > 30:
            drain *= 1.5

        # Recharge event in Warehouse
        if "Warehouse" in stage and prev < 60:
            prev = min(100, prev + np.random.uniform(5, 15))

        new_battery = max(0, prev - drain)
        battery.append(new_battery)

    df.loc[df['Transaction ID'] == tx, 'battery_level'] = battery

#  battery status
def battery_status(level):
    if level > 70:
        return "Good"
    elif level > 40:
        return "Medium"
    else:
        return "Low"

df['battery_status'] = df['battery_level'].apply(battery_status)

# Save augmented dataset
df.to_csv("smart_logistics_iot_with_battery.csv", index=False)
